### Amostragem

#### Carregamento da base de dados

In [25]:
import pandas as pd
import random 
import numpy as np

In [26]:
dataset = pd.read_csv('datasets/credit_data.csv')

In [27]:
dataset.shape

(2000, 5)

In [28]:
dataset.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [29]:
dataset.tail()

,i#clientid,income,age,loan,c#default
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0
1999,2000,69436.579552,56.152617,7378.833599,0


### Amostragem aleatória simples

In [30]:
def amostragem_aleatoria_simples(dataset, amostras):
    return dataset.sample(n = amostras)

In [31]:
df_amostra_aleatoria_simples = amostragem_aleatoria_simples(dataset, 1000)
df_amostra_aleatoria_simples.shape

(1000, 5)

### Amostragem sistemática

In [32]:
def amostragem_sistematica(dataset, amostras):
    intervalo = len(dataset) // amostras
    random.seed(1)
    inicio = random.randint(0, intervalo)
    indices = np.arange(inicio, len(dataset), step = intervalo) 
    amostra_sistematica = dataset.iloc[indices]
    return amostra_sistematica


In [33]:
df_amostra_sistematica = amostragem_sistematica(dataset, 1000)
df_amostra_sistematica.shape


(1000, 5)

### Amostragem por grupos

In [50]:
def amostragem_agrupamento(dataset, numero_grupos):
  intervalo = len(dataset) / numero_grupos
  
  grupos = []
  id_grupo = 0
  contagem = 0
  for _ in dataset.iterrows():
    grupos.append(id_grupo)
    contagem += 1
    if contagem > intervalo:
      contagem = 0
      id_grupo += 1

  dataset['grupo'] = grupos
  grupo_selecionado = random.randint(0, numero_grupos)
  return dataset[dataset['grupo'] == grupo_selecionado]

In [51]:
len(dataset) / 2

1000.0

In [52]:
df_amostra_agrupamento = amostragem_agrupamento(dataset, 2)
df_amostra_agrupamento.shape, df_amostra_agrupamento['grupo'].value_counts()

((999, 6),
 1    999
 Name: grupo, dtype: int64)

### Amostra estratificada

In [37]:
from sklearn.model_selection import StratifiedShuffleSplit

In [38]:
dataset['c#default'].value_counts()

0    1717
1     283
Name: c#default, dtype: int64

In [39]:
def amostragem_estratificada(dataset, percentual, campo):
    split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
    for _, y in split.split(dataset, dataset[campo]):
        df_y = dataset.iloc[y]
    return df_y

In [40]:
df_amostra_estratificada = amostragem_estratificada(dataset, 0.5, 'c#default')
df_amostra_estratificada.shape

(1000, 6)

### Amostra de reservatório

In [41]:
def amostragem_reservatorio(dataset, amostras):
  stream = []
  for i in range(len(dataset)):
    stream.append(i)
  
  i = 0
  tamanho = len(dataset)

  reservatorio = [0] * amostras
  for i in range(amostras):
    reservatorio[i] = stream[i]

  while (i < tamanho):
    j = random.randrange(i + 1)
    if (j < amostras): 
      reservatorio[j] = stream[i]
    i += 1

  return dataset.iloc[reservatorio]

In [42]:
df_amostragem_reservatorio = amostragem_reservatorio(dataset, 1000)
df_amostragem_reservatorio.shape

(1000, 6)

### Comparativo dos resultados

In [43]:
dataset['age'].mean(), dataset['income'].mean(), dataset['loan'].mean()

(40.80755937840458, 45331.600017793244, 4444.369694688258)

In [44]:
df_amostra_aleatoria_simples['age'].mean(), df_amostra_aleatoria_simples['income'].mean(), df_amostra_aleatoria_simples['loan'].mean()

(40.35022311632751, 44511.68778229039, 4348.662685892513)

In [45]:
df_amostra_sistematica['age'].mean(), df_amostra_sistematica['income'].mean(), df_amostra_sistematica['loan'].mean()

(40.91117381141754, 45691.49875066942, 4506.78797642633)

In [53]:
df_amostra_agrupamento['age'].mean(), df_amostra_agrupamento['income'].mean(), df_amostra_agrupamento['loan'].mean()

(40.57213154439213, 45817.42144791312, 4498.686420559128)

In [47]:
df_amostra_estratificada['age'].mean(), df_amostra_estratificada['income'].mean(), df_amostra_estratificada['loan'].mean()

(40.53363707173919, 45101.366750267334, 4423.7446510941945)

In [48]:
df_amostragem_reservatorio['age'].mean(), df_amostragem_reservatorio['income'].mean(), df_amostragem_reservatorio['loan'].mean()

(40.66004999299844, 45081.4242972876, 4429.476055841933)